In [1]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv("all_match_data_raw_copy.csv")
%run list_of_games_without_referees.ipynb

In [11]:
elevendataRefs = df['Referee'].tolist()
elevendataDataLinks = df['Data Link'].tolist()
elevendataTeams = [x.split("/")[2].split("-")[:-4] for x in elevendataDataLinks]
elevendataTeams[0:5]

[['arsenal', 'v', 'norwich', 'city'],
 ['chelsea', 'v', 'oldham', 'athletic'],
 ['coventry', 'city', 'v', 'middlesbrough'],
 ['crystal', 'palace', 'v', 'blackburn', 'rovers'],
 ['everton', 'v', 'sheffield', 'wednesday']]

In [12]:
df_fix_data = pd.read_csv("referee_fix_data.csv")

In [16]:
premdataRefs = df_fix_data['Referee'].tolist()
premdataHomeTeams = df_fix_data['HomeTeam'].tolist()
premdataAwayTeams = df_fix_data['AwayTeam'].tolist()

In [29]:
def select_home_team(match):
    homeTeam = match[0]
    if match[1] == 'v':
        return homeTeam
    elif match[2] == 'v':
        return homeTeam + " " + match[1]
    elif match[3] == 'v':
        return homeTeam + " " + match[1] + " " + match[2]
    elif match[4] == 'v':
        return homeTeam + " " + match[1] + " " + match[2] + " " + match[3]
    else:
        print("Error? : {}".format(match))
        
def select_away_team(match):
    awayTeam = match[-1]
    if match[-2] == 'v':
        return awayTeam
    elif match[-3] == 'v':
        return match[-2] + " " + awayTeam
    elif match[-4] == 'v':
        return match[-3] + " " + match[-2] + " " + awayTeam
    elif match[-5] == 'v':
        return match[-4] + " " + match[-3] + " " + match[-2] + " " + awayTeam
    else:    
        print("Error? : {}".format(match))

elevendataHomeTeams = [select_home_team(x) for x in elevendataTeams]
elevendataAwayTeams = [select_away_team(x) for x in elevendataTeams]

In [31]:
def fix_team_names(x):
    if "queens" in x:
        return "qpr"
    else:
        return x


elevendataHomeTeams = [fix_team_names(x) for x in elevendataHomeTeams]
elevendataAwayTeams = [fix_team_names(x) for x in elevendataAwayTeams]

In [35]:
j = 0
skipFirst = 6
max_j = len(premdataRefs)-1
for i in range(len(elevendataRefs)):
    eRef = elevendataRefs[i]
    eHome = elevendataHomeTeams[i]
    eAway = elevendataAwayTeams[i]
    if eRef != 'Premier League ':
        continue
    elif skipFirst > 0:
        skipFirst -= 1
        continue
    print("Fixing: {}: {} v {}".format(i,eHome,eAway))
    notfound = True
    while notfound and j <= max_j:
        pHome = premdataHomeTeams[j]
        pAway = premdataAwayTeams[j]
        print("Comparing to {} v {}".format(pHome,pAway))
        if eHome.lower() in pHome.lower() and eAway.lower() in pAway.lower():
            elevendataRefs[i] = premdataRefs[j]
            notfound = False
        j += 1
    print()

Fixing: 2159: liverpool v leicester city
Comparing to Newcastle United v Sheffield Wednesday
Comparing to Southampton v Bolton Wanderers
Comparing to Wimbledon v Liverpool
Comparing to Tottenham Hotspur v Manchester United
Comparing to Arsenal v Coventry City
Comparing to Crystal Palace v Barnsley
Comparing to Aston Villa v Blackburn Rovers
Comparing to Liverpool v Leicester City

Fixing: 2160: manchester united v southampton
Comparing to Manchester United v Southampton

Fixing: 2161: sheffield wednesday v leeds united
Comparing to Sheffield Wednesday v Leeds United

Fixing: 2162: west ham united v tottenham hotspur
Comparing to West Ham United v Tottenham Hotspur

Fixing: 2165: everton v west ham united
Comparing to Blackburn Rovers v Liverpool
Comparing to Coventry City v Bolton Wanderers
Comparing to Everton v West Ham United

Fixing: 2167: leicester city v manchester united
Comparing to Leeds United v Crystal Palace
Comparing to Leicester City v Manchester United

Fixing: 2170: tot

6

In [36]:
len(elevendataRefs)

10126

In [37]:
set(elevendataRefs)

{' Gagelmann',
 'Alan Gunn',
 'Alan Wiley',
 'Alan Wilkie',
 'Alf Buksh',
 'Andre Marriner',
 "Andy D' Urso",
 'Andy Madley',
 'Anthony Taylor',
 'Barry Knight',
 'Bob Hamer',
 'Bob Nixon',
 'Bobby Madley',
 'Brian Hill',
 'Chris Foy',
 'Chris Kavanagh',
 'Clive Wilkes',
 'Craig Pawson',
 'David Allison',
 'David Coote',
 'David Crick',
 'David Elleray',
 'David Frampton',
 'David Pugh',
 'Dermot Gallagher',
 'Eddie Wolstenholme',
 'Frazer Stretton',
 'Gary Willard',
 'Gerald Ashby',
 'Graham Barber',
 'Graham Poll',
 'Graham Scott',
 'Howard King',
 'Howard Webb',
 'I Mitchell',
 'Jeff Winter',
 'Jim Borrett',
 'Joe Worrall',
 'John E Martin',
 'John Key',
 'John Lloyd',
 'Jonathan Moss',
 'Keith A Cooper',
 'Keith Burge',
 'Keith Cooper',
 'Keith Hackett',
 'Keith Stroud',
 'Kelvin Morton',
 'Ken Redfern',
 'Keren Barratt',
 'Kevin Friend',
 'Lee Mason',
 'Lee Probert',
 'Mark Clattenburg',
 'Mark Halsey',
 'Martin Atkinson',
 'Martin Bodenham',
 'Matt Messias',
 'Michael Jones',
 'M

In [38]:
# Steve lodge = stephen lodge?
# mike riley = michael riley?
# keith cooper = keith A cooper?
df['Referee'] = elevendataRefs

In [43]:
df_gagelmann = df.query("Referee ==  ' Gagelmann'")
df_gagelmann['Data Link'].tolist()

['/matches/bolton-wanderers-v-liverpool-01-november-1997-23052/']

In [44]:
# this match turns out to have Dermot Gallagher as ref instead, so will replace
def remove_gagelmann(row):
    ref = row['Referee']
    if ref == ' Gagelmann':
        print("Found it!")
        return "Dermot Gallagher"
    else:
        return ref
df['Referee'] = df.apply(remove_gagelmann,axis=1)

Found it!


In [49]:
# Now need to investigate the missing refs
df_steve_lodge = df.query("Referee == 'Steve Lodge'")
df_stephen_lodge = df.query("Referee == 'Stephen Lodge'")
print("{} v {}".format(len(df_steve_lodge),len(df_stephen_lodge)))

157 v 18


In [48]:
df_mike_riley = df.query("Referee == 'Mike Riley'")
df_michael_riley = df.query("Referee == 'Michael Riley'")
print("{} v {}".format(len(df_mike_riley),len(df_michael_riley)))

264 v 15


In [51]:
df_keith_cooper = df.query("Referee == 'Keith Cooper'")
df_keith_a_cooper = df.query("Referee == 'Keith A Cooper'")
print("{} v {}".format(len(df_keith_cooper),len(df_keith_a_cooper)))

4 v 67


In [65]:
df_keith_a_cooper.iloc[18].tolist()[0]

'/matches/liverpool-v-wimbledon-28-december-1993-21486/'

In [66]:
def replace_ref_names(row):
    ref = row['Referee']
    if ref == 'Stephen Lodge':
        return "Steve Lodge"
    elif ref == 'Michael Riley':
        return "Mike Riley"
    elif ref == 'Keith A Cooper':
        return "Keith Cooper"
    else:
        return ref
df['Referee'] = df.apply(replace_ref_names,axis=1)

In [71]:
from collections import defaultdict
ref_dict = defaultdict(int)
for ref in df['Referee'].tolist():
    ref_dict[ref] += 1
rare_refs = defaultdict(int)
for ref in ref_dict.keys():
    count = ref_dict[ref]
    if count < 10:
        rare_refs[ref] = count
rare_refs

defaultdict(int,
            {'Ray Bigger': 4,
             'John Key': 5,
             'Bob Hamer': 2,
             'Bob Nixon': 4,
             'I Mitchell': 1,
             'Philip Wright': 4,
             'Tony Ward': 8,
             'John Lloyd': 3,
             'David Crick': 1,
             'Premier League ': 6,
             'Frazer Stretton': 1,
             'Phil Crossley': 2,
             'Richard Beeby': 3,
             'Simon Hooper': 2,
             'Andy Madley': 1,
             'David Coote': 1})

In [91]:
df_specific_ref = df.query("Referee == 'David Coote'")
df_specific_ref['Data Link'].tolist()

['/matches/newcastle-united-v-west-bromwich-albion-28-april-2018-357432/']

NameError: name 'df_specific_ref' is not defined

In [92]:
df.to_csv("all_match_data_fix1.csv",index=False)